In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv("Global_Superstore2.csv", encoding="latin1")
print(df.head())


In [ ]:
df = df[['Order Date', 'Customer ID', 'Sales']]

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [ ]:
latest_date = df['Order Date'].max() + pd.Timedelta(days=1)

In [ ]:
RFM = df.groupby('Customer ID').agg({
    'Order Date': lambda x: (latest_date - x.max()).days,   # Recency
    'Customer ID': 'count',                                # Frequency
    'Sales': 'sum'                                         # Monetary
})

In [ ]:
RFM.rename(columns={'Order Date': 'Recency',
                    'Customer ID': 'Frequency',
                    'Sales': 'Monetary'}, inplace=True)

In [ ]:
print(RFM.head())

In [ ]:
# 5. Standardize Data
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(RFM)

In [ ]:
# 6. Find Optimal Number of Clusters (Elbow Method)
wcss = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_scaled)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1, 11), wcss, marker='o')
plt.title("Elbow Method")
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42)  # choose k based on elbow graph
clusters = kmeans.fit_predict(rfm_scaled)

In [ ]:

RFM['Cluster'] = clusters

In [ ]:
# 8. Analyze Clusters
cluster_summary = RFM.groupby('Cluster').mean()
print(cluster_summary)

In [ ]:
# 9. Visualization
plt.figure(figsize=(10,6))
sns.scatterplot(x='Recency', y='Monetary', hue='Cluster', data=RFM, palette='Set2')
plt.title("Customer Segments")
plt.show()